In [ ]:
##### This code is used for obtaining the separatrix line #####
# The heuristic approach underlies in simulation of dynamical system state development with different initial conditions (values taken from a B-R grid with certain discretisation step) and observing which initial condition corresponds to atttion to either healthy or epileptic steady state
if 'RunningFromFigureScript' in locals(): # Checking where from the script was runned. If from figure, the directory path is then different
    %run ./DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
    %run ./DataGenerationScripts/dIBDRdt_functions.ipynb # loading time derivative functions 
else:
    %reset 
    %run ../DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
    %run ../DataGenerationScripts/dIBDRdt_functions.ipynb # loading time derivative functions 
    
import numpy as np
from tqdm import tqdm


In [ ]:
params=load_default_parameters('rate')
params['IBDR_E_duration']= [0,0,0,0]
params['IBDR_E_amplitude']= [0,0,0,0] 
params['Complex_input']= 'no'             
t_end= 10000
dt=1

Rmax = 0.175
Bmax = 0.175
Rmin = -0.1
Bmin = -0.1

n_disc = 170

B_grid = np.linspace(Bmin, Bmax, n_disc) # defining B-R grid 
R_grid = np.linspace(Rmin, Rmax, n_disc)
t_vect = np.linspace(0, t_end, int(t_end/dt)+1)#, retstep=True)

IBDR_4d=np.zeros([len(t_vect),n_disc,n_disc,4]) # dimensions: time, init cond B, init cond R, IBDR values

for ii in range(0,n_disc):
    for jj in range(0,n_disc):
        IBDR_4d[0,ii,jj,:]=[B_grid[ii],B_grid[ii],0,R_grid[jj]] # setting each value of the grid to be an initial condition

for tt in tqdm(range(1,len(t_vect))):
    for ii in range(0,n_disc):
        for jj in range(0,n_disc):
            IBDR_4d[tt,ii,jj,:]=IBDR_4d[tt-1,ii,jj,:]+dt*dIBDRdt_Rate(t_vect[tt],IBDR_4d[tt-1,ii,jj,0],IBDR_4d[tt-1,ii,jj,1],IBDR_4d[tt-1,ii,jj,2],IBDR_4d[tt-1,ii,jj,3], params)
            

In [ ]:
# Here, we check where did the system state developed (to epileptic or healthy attractor) for respective initial conditions. The separatrix line is obtained as a border between points leading to different outcomes
B_sep=[]
R_sep=[]
for ii in range(0,n_disc):
    for jj in range(0,n_disc):
        if (IBDR_4d[-1,ii,jj-1,1]<0.1) and (IBDR_4d[-1,ii,jj,1]>=0.1):
            B_sep.append(B_grid[ii])
            if jj==0:
                R_sep.append(R_grid[jj])
            else:
                R_sep.append((R_grid[jj]+R_grid[jj-1])/2)
      
if 'RunningFromFigureScript' in locals():
    np.save('./Data/Data_F1b_Separatrix/SeparatrixLine', [params, B_sep, R_sep, n_disc])  
else:
    np.save('../Data/Data_F1b_Separatrix/SeparatrixLine', [params, B_sep, R_sep, n_disc])  